In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import pickle as pk
import random

In [2]:
from generate_vocab import create_vocab
from src.show import show_playlist

In [3]:
collection_dir = '/media/harsh/Common/IITD/COL764-IR/Project/spotify_million_playlist_dataset/data/'
TVocab, idf = create_vocab(collection_dir)

Creating Track Vocabulary


In [4]:
# X = [[7,8,9,10],[45,67,56,57],[34,23,56,78],[68,78,89,90],[1,2,3,4],[2,3,4,5],[8,9,10,11]]
# X

In [5]:
# pca_test = PCA(n_components=2)
# for x in range(0,len(X), 2):
#     print(X[x:x+2])
#     pca_test.fit(X[x:x+2])
# print(pca_test.transform(X))

In [6]:
# from sklearn.decomposition import PCA

# for i in range(len(X)):
#     X[i] = np.array(X[i])
# pca = PCA(n_components = 2)
# pc= pca.fit(X)
# print(pc.transform(X))
# pc.transform([X[1]])

In [10]:
class create_pca:
    def __init__(self, n, vocab, collection_dir, pid_file, pca_file):
        self.n = n
        self.vocab = vocab
        self.collection_dir = collection_dir
        self.pid_file = pid_file
        self.pca_file = pca_file
    
    def get_playlist_vector(self, pid, select_subset_of = None):
        playlist = show_playlist(self.collection_dir, pid, print_op = False)
        p_vec = np.zeros(self.vocab.vocab_length)
        if playlist is None:
            return p_vec
        
        #print('retrieved pl size:',len(playlist))
        #print('for playlist:', playlist['name'], playlist['pid'])
        selected_tracks = playlist['tracks']
        if select_subset_of != None and select_subset_of < len(selected_tracks):
            print('pid', playlist['pid'])
            print('len of tracks',len(selected_tracks))
            random.seed(0)
            selected_tracks = random.sample(selected_tracks, select_subset_of)
            print('len of subset of tracks',len(selected_tracks))

        for track in selected_tracks:
            if track['track_uri'] in self.vocab.track2index and track['track_uri'] in idf:
                indx = self.vocab.to_index(track['track_uri'])
                p_vec[indx] = 1*idf[track['track_uri']]
        #print(p_vec)
        #p_vec = pca.transform(p_vec)
        return p_vec
    
    def fit_pca(self):
        pca_model = PCA(n_components=self.n)
        pids = []
        pl_vectos_x = []
        with open(self.pid_file, 'r') as f:
            pids = f.read().split()
            pids = [int(pid) for pid in pids]
        print('pids len:',len(pids))
        
        for pid in pids:
            p_vec = self.get_playlist_vector(pid = pid)
            pl_vectos_x.append(p_vec)
        pca_model.fit(pl_vectos_x)
        #print(pca_model.transform(pl_vectos_x[:2]))
        #with open('pca.pkl','rb') as f:
        
        pk.dump(pca_model, open(self.pca_file,"wb"))
        print('model saved')
        #pca_reloaded = pk.load(open(self.pca_file, 'rb'))
        #print(pca_reloaded.transform(pl_vectos_x[:2]))
        return True
    

In [11]:
c_pca = create_pca(70, TVocab, collection_dir, 'tdm_file.txt', 'pca_model_5_70.txt')

In [12]:
c_pca.fit_pca()

pids len: 4000
model saved


True

In [46]:
# pca_reloaded = pk.load('pca_model.txt')
# pca_reloaded.transform()

In [45]:
# from sklearn.decomposition import PCA
# import pickle as pk
# pca = PCA(n_components=2)
# result = pca.fit_transform(X)    
# pk.dump(pca, open("pca.pkl","wb"))


# pca_reload = pk.load(open("pca.pkl",'rb'))
# result_new = pca_reload .transform(X)
# import pickle as pk
# pca = PCA(n_components=2)
# result = pca.fit_transform(X) # Assume X is having more than 2 dimensions    
# pk.dump(pca, open("pca.pkl","wb"))

# # later reload the pickle file
# pca_reload = pk.load(open("pca.pkl",'rb'))
# result_new = pca_reload .transform([X[0]])
# print(result_new)